# Doccano JSONL to CSV Transformer
This notebook loads a Doccano-exported JSONL file of labeled segments `doccano_output_1klabeled_segments.jsonl`,
flattens the metadata fields, filters only labeled rows, and exports a clean CSV `1klabeled_segments.csv` with renamed and reordered columns.

In [33]:
import json
import pandas as pd

# Path to your Doccano JSONL export
input_path = 'doccano_output_1klabeled_segments.jsonl'
# Output CSV path
output_path = '1klabeled_segments.csv'

In [34]:
# Load JSONL into list of dicts
data = []
with open(input_path, 'r', encoding='utf-8') as f:
    for line in f:
        data.append(json.loads(line))
len(data)  # total segments, should be 7994

7994

In [35]:
# Filter only entries with a non-empty 'label' field
labeled = [d for d in data if d.get('label')]
len(labeled)  # should be 1000

1000

In [38]:
# Normalize nested 'meta' field into columns
flat_df = pd.json_normalize(labeled, sep='_')
flat_df.head()

,id,text,Comments,label,meta_datum,meta_links,meta_doc_id,meta_filter,meta_landtag,meta_beschreibung,meta_segment_id
0,8901,"24.11.2021 beimssiedlung 06:35 magdeburg, land...",[],[irrelevant],02.08.2023,[{'url': 'https://padoka.landtag.sachsen-anhal...,SA_100,[wittenberg],SA,"Aktuelle Daten im PhÃ¤nomenbereich ""Politisch ...",SA_100_8
1,10118,im vorfeld der parlamentarischen konferenz mit...,[],[irrelevant],16.11.2023,[{'url': 'https://www.parlamentsdokumentation....,BB_189,[leag],BB,,BB_189_3
2,10131,einladung: einladung 8/fin/52 ausschuss für fi...,[],[irrelevant],17.11.2023,[{'url': 'https://padoka.landtag.sachsen-anhal...,SA_232,"[wasserstoff, kraftwerke, wärmeplanung, witten...",SA,Ausschussprotokoll Ausschuss fÃ¼r Finanzen 8/F...,SA_232_1
3,9124,dokument: drucksache 8/3052 öffentlich antrag ...,[],[irrelevant],29.08.2023,[{'url': 'https://padoka.landtag.sachsen-anhal...,SA_140,[lauchstädt],SA,Gewaltschutz und Hilfssysteme im Sinne der Ist...,SA_140_1
4,11684,einladung: einladung 8/pet/39 öffentlich aussc...,[],[irrelevant],14.03.2024,[{'url': 'https://padoka.landtag.sachsen-anhal...,SA_372,[wittenberg],SA,Ausschussprotokoll Ausschuss fÃ¼r Petitionen 8...,SA_372_1


In [39]:
# Drop the Doccano-added 'id' and 'comments' column and rename fields
flat_df = flat_df.drop(columns=['id','Comments'])
flat_df = flat_df.rename(columns={
    'text':'SegmentText',
    'label':'Label',
    'meta_doc_id':'DocumentID',
    'meta_segment_id':'SegmentID',
    'meta_datum':'Date',
    'meta_beschreibung':'Description',
    'meta_landtag':'Parliament',
    'meta_links':'Links',
    'meta_filter':'Filter',
})
print(f"{flat_df.columns}")
flat_df.head()

Index(['SegmentText', 'Label', 'Date', 'Links', 'DocumentID', 'Filter',
       'Parliament', 'Description', 'SegmentID'],
      dtype='object')


,SegmentText,Label,Date,Links,DocumentID,Filter,Parliament,Description,SegmentID
0,"24.11.2021 beimssiedlung 06:35 magdeburg, land...",[irrelevant],02.08.2023,[{'url': 'https://padoka.landtag.sachsen-anhal...,SA_100,[wittenberg],SA,"Aktuelle Daten im PhÃ¤nomenbereich ""Politisch ...",SA_100_8
1,im vorfeld der parlamentarischen konferenz mit...,[irrelevant],16.11.2023,[{'url': 'https://www.parlamentsdokumentation....,BB_189,[leag],BB,,BB_189_3
2,einladung: einladung 8/fin/52 ausschuss für fi...,[irrelevant],17.11.2023,[{'url': 'https://padoka.landtag.sachsen-anhal...,SA_232,"[wasserstoff, kraftwerke, wärmeplanung, witten...",SA,Ausschussprotokoll Ausschuss fÃ¼r Finanzen 8/F...,SA_232_1
3,dokument: drucksache 8/3052 öffentlich antrag ...,[irrelevant],29.08.2023,[{'url': 'https://padoka.landtag.sachsen-anhal...,SA_140,[lauchstädt],SA,Gewaltschutz und Hilfssysteme im Sinne der Ist...,SA_140_1
4,einladung: einladung 8/pet/39 öffentlich aussc...,[irrelevant],14.03.2024,[{'url': 'https://padoka.landtag.sachsen-anhal...,SA_372,[wittenberg],SA,Ausschussprotokoll Ausschuss fÃ¼r Petitionen 8...,SA_372_1


In [40]:
# Reorder & export
cols = ['DocumentID','SegmentID','Date','Parliament','Description','Links','Filter','SegmentText','Label']
flat_df[cols].to_csv(output_path, index=False)
print(f'Exported to {output_path}')

Exported to 1klabeled_segments.csv
